In [0]:
import glob
import openpyxl
import pandas as pd

appended_project_data = []
appended_results_data = []

path = "/dbfs/mnt/geochem/samples/*"
files = glob.glob(path)

for file in files:
  print(file)
  wb = openpyxl.load_workbook(file)
  sheets = wb.sheetnames
  
  for sheet in sheets:
    print(sheet)
    
    #read the data sheet into a dataframe
    ws = wb[sheet]
    data = ws.values
    columns = next(data)[0:]
    initial_wheader_df = pd.DataFrame(data, columns=columns)
    
    #add the sheename as a column in the dataframe (representing the sample)
    initial_wheader_df['Sample Type']=sheet
    initial_wheader_df = initial_wheader_df[initial_wheader_df['Sample ID'].notnull()]

    #preserve original initial copy of the dataframe
    full_copy_df = initial_wheader_df.copy()
    Project_df =  full_copy_df[['Sample ID','Project Name','Sample Type','Material Type','Lithology','Zone','Borehole']]

    #create a dataframe with the tombstone info
    columns = ['Project Name','Sample Type','Material Type','Lithology','Zone','Borehole']
    results_df = full_copy_df.copy()
    results_df.drop(columns, inplace=True, axis=1)
    
    #unpivot the remaining columns as Test and results using the Sample ID as the index
    results_df = pd.melt(results_df, id_vars=[('Sample ID')],var_name='Test')

    #ensure the Sample ID is unique 
    assert results_df['Sample ID'].duplicated().any()==False
    assert Project_df['Sample ID'].duplicated().any()==False
    
    #keep appending the dataframe to previously loaded dataframes (at the sheet level)
    appended_results_data.append(results_df)
    appended_project_data.append(Project_df)

#keep appending the dataframe to previously loaded dataframes (at the excel file level)
appended_results_df = pd.concat(appended_results_data)
appended_project_df = pd.concat(appended_project_data)
 
#finallly join the two dataframes (projects and results) based on sample id
final_df=pd.merge(appended_project_df,appended_results_df,on='Sample ID', how='left')


/dbfs/mnt/geochem/samples/Beaver Dam_ABASFETRM_Sharing.xlsx
TRM_aquaregia
ABA
SFE
/dbfs/mnt/geochem/samples/Brucejack _ABATRMSFE_Sharing.xlsx
ABA
TRM_aquaregia
SFE
/dbfs/mnt/geochem/samples/Cote_ABATRMNAGXRFSFESPLP_Sharing.xlsx
ABA
TRM_aquaregia
TRM_4acid
NAG
XRF
SFE
SPLP

In [0]:
final_df

Out[2]:

,Sample ID,Project Name,Sample Type,Material Type,Lithology,Zone,Borehole,Test,value
0,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Start_m,149
1,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Finish_m,150
2,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Length_m,None
3,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Grain Size,None
4,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Ag_mg/kg,-0.2
...,...,...,...,...,...,...,...,...,...
961704,CND1(CN-18MC-13-03),Côté,SPLP,Tailings,Composite,None,None,SPLP-Zn_mg/L,-0.001
961705,CND1(CN-18MC-13-03),Côté,SPLP,Tailings,Composite,None,None,SPLP-Zr_mg/L,None
961706,CND1(CN-18MC-13-03),Côté,SPLP,Tailings,Composite,None,None,SPLP-NH3_mg/L,None
961707,CND1(CN-18MC-13-03),Côté,SPLP,Tailings,Composite,None,None,SPLP-NH4_mg/L,None


####Convert the Pandas Dataframe to Spark Dataframe for downstream processing

In [0]:
final_df[['Sample ID','Project Name','Sample Type','Material Type','Lithology','Zone','Borehole','Test','value']] = final_df[['Sample ID','Project Name','Sample Type','Material Type','Lithology',	'Zone','Borehole','Test','value']].astype(str)

In [0]:
spark_final_results_df = spark.createDataFrame(final_df)

####Create a temporary view from the dataframe

In [0]:
spark_final_results_df.createOrReplaceTempView("tempvwresults")

In [0]:
final_df

Out[82]:

,Sample ID,Project Name,Sample Type,Material Type,Lithology,Zone,Borehole,Test,value
0,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Start_m,149
1,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Finish_m,150
2,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Length_m,None
3,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Grain Size,None
4,LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Ag_mg/kg,-0.2
...,...,...,...,...,...,...,...,...,...
961704,SU-114-001,Brucejack,SFE,WasteRock,SilicifiedBreccia,TheValleyofKings(VOK)OreZone,None,SFE-Zn_mg/L,-0.001
961705,SU-114-001,Brucejack,SFE,WasteRock,SilicifiedBreccia,TheValleyofKings(VOK)OreZone,None,SFE-Zr_mg/L,1e-05
961706,SU-114-001,Brucejack,SFE,WasteRock,SilicifiedBreccia,TheValleyofKings(VOK)OreZone,None,SFE-NH3_mg/L,None
961707,SU-114-001,Brucejack,SFE,WasteRock,SilicifiedBreccia,TheValleyofKings(VOK)OreZone,None,SFE-NH4_mg/L,None


####Store the data into a delta lake table

In [0]:
%sql
TRUNCATE TABLE GEOCHEM.PROJECT_SAMPLE_TEST_RESULT;

INSERT INTO GEOCHEM.PROJECT_SAMPLE_TEST_RESULT (SELECT * FROM tempvwresults)

####Use SQL to analyze the data

In [0]:
%sql
SELECT distinct PROJECT_NM,SAMPLE_TYPE_LBL FROM GEOCHEM.PROJECT_SAMPLE_TEST_RESULT;


PROJECT_NM,SAMPLE_TYPE_LBL
BeaverDam,ABA
Côté,ABA
BeaverDam,SFE
BeaverDam,TRM_aquaregia
null,null
Côté,TRM_4acid
Côté,NAG
Côté,TRM_aquaregia
Côté,SFE
Côté,SPLP


In [0]:
%sql
SELECT * FROM GEOCHEM.PROJECT_SAMPLE_TEST_RESULT 

SAMPLE_ID,Project_NM,Sample_Type_LBL,Material_Type_LBL,Lithology_LBL,Zone_LBL,Borehole_LBL,Test_NM,Value_SRT
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Start_m,149.0
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Finish_m,150.0
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Length_m,None
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,Grain Size,None
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Ag_mg/kg,-0.2
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Al_mg/kg,27400.0
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Au_mg/kg,None
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-As_mg/kg,17.0
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-B_mg/kg,-10.0
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Ba_mg/kg,90.0


In [0]:
%sql
select Project_NM,count(*) from GEOCHEM.PROJECT_SAMPLE_TEST_RESULT where Project_NM is not null
group by Project_NM
Order by count(*) 

Project_NM,count(1)
BeaverDam,22883
Brucejack,227352
Côté,711474


In [0]:
%sql
SELECT * FROM GEOCHEM.PROJECT_SAMPLE_TEST_RESULT
WHERE SAMPLE_TYPE_LBL ='TRM_aquaregia' 
  AND Test_NM = 'AR-Fe_mg/kg'
  AND Value_SRT > 1700


SAMPLE_ID,Project_NM,Sample_Type_LBL,Material_Type_LBL,Lithology_LBL,Zone_LBL,Borehole_LBL,Test_NM,Value_SRT
LX17-12,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Fe_mg/kg,45700.0
LX17-22,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Fe_mg/kg,24800.0
LX17-24,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE,None,BD14-186,AR-Fe_mg/kg,48500.0
LX17-02,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD15-GT02,AR-Fe_mg/kg,62200.0
LX17-03,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD15-GT02,AR-Fe_mg/kg,53000.0
LX17-04,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD14-186,AR-Fe_mg/kg,61600.0
LX17-09,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD14-160,AR-Fe_mg/kg,50900.0
LX17-10,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD14-160,AR-Fe_mg/kg,44500.0
LX17-11,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD14-160,AR-Fe_mg/kg,54000.0
LX17-13,BeaverDam,TRM_aquaregia,MineRock,ARGILLITE-GREYWACKE,None,BD14-186,AR-Fe_mg/kg,48400.0


In [0]:
%sql

CREATE SCHEMA GEOCHEM;

CREATE TABLE GEOCHEM.PROJECT_SAMPLE_TEST_RESULT (
SAMPLE_ID string,
Project_NM string,
Sample_Type_LBL string,
Material_Type_LBL string,
Lithology_LBL string,
Zone_LBL string,
Borehole_LBL string,
Test_NM string,
Value_SRT string
);